In [1]:
import tensorflow as tf
tf.__version__

'2.4.1'

###**Training Callback**

In [2]:
from tensorflow.keras.callbacks import Callback

class TrainingCallback(Callback):
  # To overwritten the below functions
  
  def on_train_begin(self, logs=None):
    print('Traning starts...')

  def on_epoch_begin(self, epoch, logs=None):
    print(f'Starting epoch {epoch}')

  def on_train_batch_begin(self, batch, logs=None):
    print(f'Starting batch {batch}')

  def on_train_batch_end(self, batch, logs=None):
    print(f'Finishing batch {batch}')

  def on_epoch_end(self, epoch, logs=None):
    print(f'Finishing epoch {epoch}')

  def on_train_end(self, logs=None):
    print(f'Finishing training!')

###**Testing Callback**

In [3]:
class TestingCallback(Callback):
  # To overwritten the below functions

  def on_test_begin(self, logs=None):
    print('Testing starts...')
  
  def on_test_batch_begin(self, batch, logs=None):
    print(f'Testing: Starting batch {batch}')

  def on_test_batch_end(self, batch, logs=None):
    print(f'Testing: Finishing batch {batch}')

  def on_test_end(self, logs=None):
    print(f'Testing: Finished!')

### **Prediction Callback**

In [20]:
class PredictionCallback(Callback):
  # To overwritten the below functions

  def on_predict_begin(self, logs=None):
    print('Prediction starts...')
  
  def on_predict_batch_begin(self, batch, logs=None):
    print(f'prediction: Starting batch {batch}')

  def on_predict_batch_end(self, batch, logs=None):
    print(f'Prediction: Finishing batch {batch}')

  def on_predict_end(self, logs=None):
    print(f'Prediction: Finished!')

In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

###**Validation sets**

In [6]:
from sklearn.datasets import load_diabetes

diabetes_dataset = load_diabetes()
#print(diabetes_dataset['DESCR'])

In [7]:
diabetes_dataset.keys()

dict_keys(['data', 'target', 'DESCR', 'feature_names', 'data_filename', 'target_filename'])

**Get data and target**

In [8]:
data = diabetes_dataset['data']
targets = diabetes_dataset['target']

In [9]:
# targets

**Normalizes the target**

In [10]:
targets = (targets - targets.mean(axis=0)) / targets.std()
# targets

**Split data into Train and Test sets**

In [11]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_targets, test_targets = train_test_split(data, targets, test_size=0.1)

In [12]:
print(train_data.shape)
print(test_data.shape)
print(train_targets.shape)
print(test_targets.shape)

(397, 10)
(45, 10)
(397,)
(45,)


### **Train a Feedforward NN**

In [13]:
from tensorflow.keras.layers import Dropout
from tensorflow.keras import regularizers

In [14]:
def get_regularized_model(weight_decay, dropout_rate):
  model = Sequential([
    Dense(128, kernel_regularizer=regularizers.l2(weight_decay), activation='relu', input_shape=(train_data.shape[1],)),
    Dropout(dropout_rate),
    Dense(128, kernel_regularizer=regularizers.l2(weight_decay), activation='relu'),
    Dropout(dropout_rate),
    Dense(128, kernel_regularizer=regularizers.l2(weight_decay), activation='relu'),
    Dropout(dropout_rate),
    Dense(128, kernel_regularizer=regularizers.l2(weight_decay), activation='relu'),
    Dropout(dropout_rate),
    Dense(128, kernel_regularizer=regularizers.l2(weight_decay), activation='relu'),
    Dropout(dropout_rate),
    Dense(128, kernel_regularizer=regularizers.l2(weight_decay), activation='relu'),
    Dropout(dropout_rate),
    Dense(1)])
  return model

model = get_regularized_model(weight_decay=1e-5, dropout_rate=0.3)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               1408      
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               16512     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               1

In [16]:
model.compile(optimizer='adam', loss='mse')

**Training model with Callback**

In [17]:
history = model.fit(train_data, 
                    train_targets, 
                    epochs=3,
                    validation_split=0.15,
                    batch_size=128,
                    verbose=False,
                    callbacks=[TrainingCallback()]
                    )
print(history.history.keys())  # dict_keys(['loss', 'mape', 'val_loss', 'val_mape'])

Traning starts...
Starting epoch 0
Starting batch 0
Finishing batch 0
Starting batch 1
Finishing batch 1
Starting batch 2
Finishing batch 2
Finishing epoch 0
Starting epoch 1
Starting batch 0
Finishing batch 0
Starting batch 1
Finishing batch 1
Starting batch 2
Finishing batch 2
Finishing epoch 1
Starting epoch 2
Starting batch 0
Finishing batch 0
Starting batch 1
Finishing batch 1
Starting batch 2
Finishing batch 2
Finishing epoch 2
Finishing training!
dict_keys(['loss', 'val_loss'])


### **Evaluate with testing callback**

In [18]:
model.evaluate(test_data, test_targets, verbose=2, callbacks=[TestingCallback()])

Testing starts...
Testing: Starting batch 0
Testing: Finishing batch 0
Testing: Starting batch 1
Testing: Finishing batch 1
2/2 - 0s - loss: 0.7792
Testing: Finished!


0.7791553139686584

**Prediction with callback**

In [21]:
model.predict(test_data, verbose=False, callbacks=[PredictionCallback()])

Prediction starts...
prediction: Starting batch 0
Prediction: Finishing batch 0
prediction: Starting batch 1
Prediction: Finishing batch 1
Prediction: Finished!


array([[0.03108064],
       [0.06108658],
       [0.05500937],
       [0.01261233],
       [0.02764589],
       [0.07542811],
       [0.03734202],
       [0.0294998 ],
       [0.02394306],
       [0.01459122],
       [0.06514514],
       [0.07170716],
       [0.02938245],
       [0.03741388],
       [0.03888236],
       [0.01177666],
       [0.08260433],
       [0.05892004],
       [0.04720611],
       [0.07782611],
       [0.06491879],
       [0.03731585],
       [0.0147785 ],
       [0.01154304],
       [0.04833364],
       [0.0641339 ],
       [0.0111873 ],
       [0.0432971 ],
       [0.08781214],
       [0.01266828],
       [0.06444703],
       [0.00980456],
       [0.02932644],
       [0.00836737],
       [0.04804941],
       [0.00953231],
       [0.04342889],
       [0.05920814],
       [0.04741011],
       [0.01329505],
       [0.06311829],
       [0.02281408],
       [0.04809755],
       [0.02499654],
       [0.01887225]], dtype=float32)